In [1]:
import pandas as pd
import seaborn as sns
from typing import Tuple
from scipy.stats import mode
from sklearn.metrics import confusion_matrix

class KNNClassifier:


    def __init__(self,k:int,test_split_ratio:float):
        self.k = k
        self.test_split_ratio = test_split_ratio
        self.x_train = 0
        self.y_train = 0
        self.x_test = 0
        self.y_test = 0